In [ ]:
!pip install transformers
!pip install light-the-torch
!ltt install torch torchvision
!pip install sacremoses
!pip install gradio
!pip install accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Obtaining dependency information for nvidia-cuda-nvrtc-cu12==12.1.105 from https://files.pythonhosted.org/packages/b6/9f/c64c03f49d6fbc56196664d05dba14e3a561038a81a638eeb47f4d4cfd48/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for nvidia-cuda-runtime-cu12==12.1.105 from https://files.pythonhosted.org/packages/eb/d5/c68b1d2cdfcc59e72e8a5949a37ddb22ae6cade80cd4a57a84d4c8b55472/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for nvidia-cuda-cupti-cu12==12.1.105 from https://files.pythonhosted.org/packages/7e/00/6b218edd739ecfc60524e585ba8e6b00554dd908de2c9c66c1af3e44e18d/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Obtain

In [ ]:
!pip install gradio_client

### ChatDoctor


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import gradio as gr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "Narrativaai/BioGPT-Large-finetuned-chatdoctor"

tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")

model = AutoModelForCausalLM.from_pretrained(model_id)


# Move the model to the device
model = model.to(device)

def answer_question(
        prompt,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=2,
        **kwargs,
):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Move the inputs to the device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id

        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    return output.split(" Response:")[1]


def gui_interface(prompt):
    prompt="""
    Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.

    ### Instruction:
    If you are a doctor, please answer the medical questions based on the patient's description.

    ### Input:
    """+prompt+"""
    ### Response:
    """
    return answer_question(prompt)

iface = gr.Interface(fn=gui_interface, inputs="text", outputs="text")
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://419b122f18b0c8cb99.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from gradio_client import Client

client = Client("https://6b2871c91eb9ebc65e.gradio.live/")
result = client.predict(
		"Hello i have headache",
		api_name="/predict"
)
print(result)

Loaded as API: https://6b2871c91eb9ebc65e.gradio.live/ ✔


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Hello! Welcome and thank you for asking on Chat Doctor! I understand your concern and would explain that your symptoms could be related to tension type headache. For this reason, I recommend consulting with a neurologist for a careful physical exam and some tests: - a brain MRI-complete blood count, PCR, ESR for inflammation- thyroid hormone levels for thyroid gland dysfunction- blood electrolytes for possible imbalance. Hope you will find this answer helpful! Kind regards,


## Gradio

# Upload from Drive

In [ ]:
import requests

url = "https://huggingface.co/Narrativaai/BioGPT-Large-finetuned-chatdoctor/resolve/main/model.safetensors?download=true"
local_filename = "model.safetensors"

i=0
chunk_size = 1024 * 1024
response = requests.get(url, stream=True)
with open(local_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=chunk_size):
        i=i+1
        if i%100==0: print(i)
        f.write(chunk)

print(f"file {local_filename} downloaded successfully!")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
file model.safetensors downloaded successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r ./drive/MyDrive/DoctorModel/ ./

In [ ]:
#same code without Gradio and By Getting Model from Drive
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

local_model_path = "./DoctorModel"

tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")


model = AutoModelForCausalLM.from_pretrained(local_model_path)


model = model.to(device)

def answer_question(
        prompt,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=2,
        **kwargs,
):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Move the inputs to the device
    inputs = {key: val.to(device) for key, val in inputs.items()}
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id

        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    return output.split(" Response:")[1]


def gui_interface(prompt):
    prompt="""
    Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.

    ### Instruction:
    If you are a doctor, please answer the medical questions based on the patient's description.

    ### Input:
    """+prompt+"""
    ### Response:
    """
    return answer_question(prompt)

print(gui_interface("I have headache and cough"))


cuda


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configurati

 Hello! Welcome and thank you for asking on Chat Doctor! I understand your concern and would explain that your symptoms seem to be related to a viral infection. I recommend consulting with your GP for a careful physical examination and some tests: - a chest X-ray study-complete blood count-PCR, ESR for inflammation- blood electrolytes- kidney and liver function tests. Hope you will find this answer helpful! Kind regards,
